-- Initial questions

Jonas Barros

**OBSERVATION: Some cell results have been removed due to sensitive data that cannot be shared. The code remains unaltered.**

# Connecting to database

In [61]:
# Importing libraries
import numpy as np
import pandas as pd
#import seaborn as sns
import os
#import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

from sqlalchemy import create_engine

# Getting connection
def get_db_conn():
    """ Get an authenticated psycopg db connection"""
    
    user=os.getenv('PGUSER'),  # returns tuple
    password=os.getenv('PGPASSWORD'), #returns tuple
    host=os.getenv('PGHOST'), #returns tuple
    port=int(os.getenv('PGPORT')), #returns tuple
    database=os.getenv('PGDATABASE')
    
    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user[0], 
                                                                password[0], 
                                                                host[0], 
                                                                port[0], 
                                                                database))
    connection = engine.connect()
    
    return connection

db_conn = get_db_conn()

# Initial questions

## What are the number of emergency demolitions vs. number of demolitions?

Relevant tables:
- completedcitydemolition
- planneddemolition

Both tables share an 'id_demo_rf' column. Let's join the tables on this column and see if their content seems to match (i.e., if both columns share the same information).

In [2]:
q = '''
SELECT
    *
FROM
    raw.completedcitydemolition c
LEFT JOIN raw.planneddemolition p ON
    c.id_demo_rf = p.id_demo_rf
'''

results = pd.read_sql(q, db_conn)

In [ ]:
results

In [4]:
#Filter all None values from dataframe on column collapsedg
results = results.dropna(subset=['collapsedg'])

In [5]:
results

,objectid,id_demo_rf,groupstatu,decision,core_phase,shape_length,shape_area,shape,objectid,id_demo_rf,groupstatu,decision,core_phase,collapsedg,shape_length,shape_area,shape


As we can see, there is no match between the two tables, which lead us to conclude that the planned demolition table (with just 168 rows) may not keep any record of previous planned demolitions, serving solely as a snapshot of currently planned demolitions.

It is also worth noting that, while the completed demolition table contains much more registries, it basically provides no useful information beyond columns "groupstatu", "decision" and "core_phase".

Let's take a look at the unique values of these columns and how they interact between themselves.

In [6]:
del results

q = '''
SELECT
    DISTINCT groupstatu, decision, core_phase
FROM
    raw.completedcitydemolition
'''

results = pd.read_sql(q, db_conn)

In [7]:
# For each column, return their unique values
uniques = results.apply(lambda x: x.unique())
n_uniques = results.apply(lambda x: x.nunique())

In [8]:
n_uniques.to_frame()

,0
groupstatu,1
decision,6
core_phase,9


In [9]:
with pd.option_context('display.max_colwidth', None):
    display(uniques.to_frame())

,0
groupstatu,[Final/Full Demo Complete]
decision,"[47, 90, 124, 48, 49, 91]"
core_phase,"[ , Removed from 5- City to Demo, 1, NA, 2, 3, Removed from 5 - City to demo, 4, removed from CORE Phase 5 to move to CDBG]"


In [ ]:
results

We can see that the "groupstatu" column does not provide any useful information since it contains the same value for all rows. This might be an indicative that this table is an extraction of a bigger, more detailed table.

The decision column seems to be the most useful since it contains six different values. However, we do not know their meaning. A dictionary for this data need to be requested.

Lastly, the "core_phase" column actually contains only six unique values, instead of nine. The columns with the "5" number are actually the same content, with a typing difference (the space before the dash character), and there are both null values and empty fields. It does provide some useful insights on the core program, which apparently has 5 different phases and at least some vacant lots (considering that this list contains demolitions that occured) are moved to the CDBG: Community Development Block Grant Programs.

Since we are already here, let's just take a look at how the data is distributed across the columns.

In [11]:
del results

q = '''
SELECT
    decision, count(*)
FROM
    raw.completedcitydemolition
GROUP BY
    decision
'''

results = pd.read_sql(q, db_conn)

In [ ]:
results

In [13]:
del results

q = '''
SELECT
    core_phase, count(*)
FROM
    raw.completedcitydemolition
GROUP BY
    core_phase
'''

results = pd.read_sql(q, db_conn)

In [ ]:
results

So basically the decisions with codes 90 and 47 are the ones that concentrate most of the results. Meanwhile, on the core_phase side, most rows are empty, with group 4 having a slightly majority among those rows with a core_phase.

### TL;DR
> 1. We need to request a dictionary to better understand what the 'decision' and 'core_phase' codes mean.

> 2. We need to ask if the completeddemolition table is just an extraction of a bigger, more relevant table.

> 3. We need to ask if there is any historical data on planned demolitions.

> 4. Data on demolition currently is to scarce

## How to connect properties in the tax parcel database and real estate data?

The real state table does not contain much information by itself beyond date of deed, price and neighborhood, so it need to check if it has an unique id that can be crossed with potentially other useful data.

Let's do that now. There are three possibilities that can be used:
1. ogc_fid
2. blocklot
3. objectid

Testing the first one:

In [15]:
del results

# Select tables that contain a column with 'ogc' in the name
q = '''
SELECT
    *
FROM
    information_schema.columns
WHERE
    table_schema = 'raw'
    AND column_name LIKE '%%ogc%%';
'''

results = pd.read_sql(q, db_conn)

In [16]:
results

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,baltimore-roofs,raw,open_notice_vacant,ogc_fid,1,nextval('raw.open_notice_vacant_ogc_fid_seq'::...,NO,integer,None,None,32,2,0,None,None,None,None,None,None,None,None,None,None,None,None,baltimore-roofs,pg_catalog,int4,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
1,baltimore-roofs,raw,real_estate_data,ogc_fid,1,nextval('raw.real_estate_data_ogc_fid_seq'::re...,NO,integer,None,None,32,2,0,None,None,None,None,None,None,None,None,None,None,None,None,baltimore-roofs,pg_catalog,int4,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
2,baltimore-roofs,raw,building_construction_permits,ogc_fid,1,nextval('raw.building_construction_permits_ogc...,NO,integer,None,None,32,2,0,None,None,None,None,None,None,None,None,None,None,None,None,baltimore-roofs,pg_catalog,int4,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
3,baltimore-roofs,raw,data_311,ogc_fid,1,nextval('raw.data_311_ogc_fid_seq'::regclass),NO,integer,None,None,32,2,0,None,None,None,None,None,None,None,None,None,None,None,None,baltimore-roofs,pg_catalog,int4,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
4,baltimore-roofs,raw,tax_parcel_address,ogc_fid,1,nextval('raw.tax_parcel_address_ogc_fid_seq'::...,NO,integer,None,None,32,2,0,None,None,None,None,None,None,None,None,None,None,None,None,baltimore-roofs,pg_catalog,int4,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES


The "tax_parcel_addres" seems to be the table with more potential to provide any insights, so let's take a look at it:

In [17]:
del results

# Join real state data and tax parcel address tables
q = '''
SELECT
    *
FROM
    raw.real_estate_data red
LEFT JOIN raw.tax_parcel_address tpa ON
    red.ogc_fid = tpa.ogc_fid;
'''
results = pd.read_sql(q, db_conn)

In [ ]:
results

That's too much data, let's refine our results a little bit for better visual comparison:

In [39]:
del results

q = '''
SELECT
    red.ogc_fid ,
    red.objectid ,
    red.blocklot ,
    red.nhood ,
    tpa.ogc_fid,
    tpa.objectid ,
    tpa.blocklot ,
    tpa.neighbor
FROM
    raw.real_estate_data red
LEFT JOIN raw.tax_parcel_address tpa ON
    red.ogc_fid = tpa.ogc_fid;
'''
results = pd.read_sql(q, db_conn)

In [ ]:
results

Blocklot is not matching for most of the rows, the same for the neighborhoods. This lead us to believe that each data has its own ID and it is not possible to cross them using the ogc_fid column. However, what if we use the blocklot column?

In [42]:
del results

q = '''
SELECT
    red.ogc_fid ,
    red.objectid ,
    red.blocklot ,
    red.nhood ,
    tpa.ogc_fid,
    tpa.objectid ,
    tpa.blocklot ,
    tpa.neighbor
FROM
    raw.real_estate_data red
LEFT JOIN raw.tax_parcel_address tpa ON
    red.blocklot = tpa.blocklot;
'''
results = pd.read_sql(q, db_conn)

In [ ]:
results

It does seem to work, at least for some of the rows. Let's check how many that is.

In [ ]:
# Convert all cells in nhood to uppercase and remove all non-alphanumeric characters
results['nhood_cleaned'] = results['nhood'].str.upper().str.replace('[^A-Za-z]+', '')
results['neighbor_cleaned'] = results['neighbor'].str.upper().str.replace('[^A-Za-z]+', '')


In [50]:
# Filtering only rows that do not match nhood column to neighbor column
results_incompatible = results[results['nhood_cleaned'] != results['neighbor_cleaned']]

In [ ]:
results_incompatible

In [46]:
unmatches = results_incompatible[['nhood_cleaned','neighbor_cleaned']].drop_duplicates()

# Remove rows with empty value
unmatches = unmatches[~unmatches['nhood_cleaned'].str.contains('^$')]

# Remove rows with null value
unmatches = unmatches[~unmatches['neighbor_cleaned'].isnull()]

In [47]:
u_unmatches = unmatches.drop_duplicates()

In [48]:
print(f'Unmatched pairs: {len(u_unmatches)} \nUnmatched rows:{len(unmatches)}')

Unmatched pairs: 28 
Unmatched rows:28


There are 28 exceptional cases where the neighborhood is not matching when joinig by the blocklot.
Let's take a look at a few of those.
For row '10483' the neighborhood in real_state_data is 'Inner Harbor' while the neighborhood in the tax_parcel_address is 'Fells Point'. However, these two neighborhoods are very close, so it could be a grey area situation.

However, if we look at row '15844' the conflicting neighborhoods are West Arlington and Uplands, which are considerable distant. The tax parcel data has both the address and the geometry data for it so we are able to check if at least there is internal cohesion to the database.


In [24]:
# Getting coordinates from the wkb_geometry column

q = '''
SELECT
blocklot,
ST_AsText(ST_transform(ST_centroid(wkb_geometry), 4326))
FROM
    raw.tax_parcel_address red
WHERE
    blocklot = '2550A008H';
'''
results = pd.read_sql(q, db_conn)

In [ ]:
results

Looking at [Google Maps](https://www.google.com.br/maps/place/39%C2%B017'22.8%22N+76%C2%B041'23.3%22W/@39.2896746,-76.6920018,17z) we can see that the address provided by the tax parcel data is the same contained in the wkb geometry column and both are located in Edmondson Village / Uplands. The real state table, therefore, seems to be more trustworthy in terms of neighborhood, but at least we can assume that they are both talking about the same property, even though the tax parcel has the wrong neighborhood in it.

Also worth noting the neighborhood column in the tax parcel database contains more than one spelling for the same neighborhood, revealing less consistency. For example, in the tax parcel table we have both "SBI" and "South Baltimore" as neighborhood names, even though they mean the same.

Comparing unique values from both tables:

In [60]:
q = '''
SELECT DISTINCT
    red.nhood ,
    tpa.neighbor
FROM
    raw.real_estate_data red
LEFT JOIN raw.tax_parcel_address tpa ON
    red.blocklot = tpa.blocklot;
'''
results = pd.read_sql(q, db_conn)

un_tax = results[['neighbor']].drop_duplicates().sort_values(by = 'neighbor')
un_real = results[['nhood']].drop_duplicates()

In [58]:
print(f'Unique neighborhoods in tax_parcel database: {len(un_tax)}\nUnique neighborhoods in real_estate_data database: {len(un_real)}')

Unique neighborhoods in tax_parcel database: 272
Unique neighborhoods in real_estate_data database: 265


On the other hand, it is important to keep in mind that some columns are empty on one table but not on another, and vice-versa:

In [69]:
# Substitute rows with "0" value to null
results['nhood'] = results['nhood'].replace('0', np.nan)
results['neighbor'] = results['neighbor'].replace('0', np.nan)
# Count rows with null value in column neighbor
print(f'''Empty rows in tax_parcel: {len(results[results['neighbor'].isnull()])}
Empty rows in real_estate: {len(results[results['nhood'].isnull()])}''')

# Count rows with null value in both columns neighbor and nhood
print(f'''Empty rows in both: {len(results[results['neighbor'].isnull() & results['nhood'].isnull()])}''')

Empty rows in tax_parcel: 121
Empty rows in real_estate: 78
Empty rows in both: 1


So while the real estate data seem more reliable, we should keep in mind that it might need to be complemented by the real state data.

### TL;DR
> 5. ogc_fid and objectid does not match between tax parcel data and real estate data, suggesting it could be an unique identifier for each table.

> 6. Blocklot seems to be a reliable way to connect tax parcel and real estate data.

> 7. Both tables have missing fields, with real estate data apparently being more reliable.